In [ ]:
import pandas as pd
import os

In [ ]:
# Create folders to mount
! mkdir working

# Read/Write Access
! gcsfuse --dir-mode 777 --file-mode 777 --implicit-dirs gp2_working_eu working

In [ ]:
wd = '/YOUR/WORKING/DIR'

### Match Variants

In [ ]:
# Normalized variant files

label = 'AJ'
user = 'nicole'

In [ ]:
# Find intersection of variants - needs some manual cleaning/code editing to exclude extra words

for region in ['non_PAR', 'PAR']:
    male = f'{wd}/working/{user}/XWAS/{label}/{label}_{region}_male_normalized'
    female = f'{wd}/working/{user}/XWAS/{label}/{label}_{region}_female_normalized'

    ! awk 'NR>1{print $3}' {female}.pvar | sort > female_{region}.ids
    ! awk 'NR>1{print $3}' {male}.pvar | sort > male_{region}.ids
    ! comm -12 female_{region}.ids male_{region}.ids > common_{region}.ids

In [ ]:
os.chdir('/dev/shm')

for region in ['non_PAR', 'PAR']:
    male = f'{wd}/working/{user}/XWAS/{label}/{label}_{region}_male_normalized'
    female = f'{wd}/working/{user}/XWAS/{label}/{label}_{region}_female_normalized'

    ! plink2 --pfile {female} \
        --extract {wd}/common_{region}.ids \
        --make-pgen \
        --sort-vars \
        --out {female}_common_vars

    ! plink2 --pfile {male} \
        --extract {wd}/common_{region}.ids \
        --make-pgen \
        --sort-vars \
        --out {input}_common_vars

### Sex-Stratified Region-Stratified XWAS

In [ ]:
region = 'PAR'
sex = 'male'

input = f'{wd}/working/{user}/XWAS/{label}/{label}_{region}_{sex}_normalized_common_vars'

In [ ]:
# Run command
"""
GLM Modes:
    - xchr-model 1. Male dosages are on a 0..1 scale on chrX, while females are 0..2. This is PLINK 1.9 default.
    - xchr-model 2. Males and females are both on a 0..2 scale on chrX. This is the PLINK 2 default.
PC selection based on Scree Plot in 02_gwas_prep.ipynb
"""

gwas_cmd = f'plink2 --pfile {input} \
--glm hide-covar hide-covar firth-fallback no-x-sex cols=+a1freq,+a1count \
--xchr-model 2 \
--pheno-name PHENO \
--pheno {wd}/prep_files/{sex}_{label}_pheno.txt \
--covar {wd}/prep_files/{sex}_{label}_covar.txt \
--covar-name PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8 \
--covar-variance-standardize \
--out {wd}/working/{user}/XWAS/{label}/XWAS_{label}_{region}_{sex}'

# Launch in /dev/shm
! {gwas_cmd}